# 目的
CryptoCompareから取得されたデータを用いていろいろやってみる。

# getRateViaCrypto()
CryptoCopmareからデータを取得する。

In [ ]:
import urllib.request
import urllib.parse
import json

def getRateViaCrypto(histoticks, params):
    """getRateViaCrypto(histoticks, params)
    get rate via CryptoCompare (https://min-api.cryptocompare.com/).
    params should be a dict object with items of str.
    <params>
    histoticks: day, hour, minute or some tick (see the site)
    params: parameters to send the site 
        fsym : currency symbol of interest (required)
        tsym : currency symbol to convert into (required)
        limit: limit of retrieved data (max: 2000)
        e    : the exchange to obtain data from (CCCAGG - by default)
        toTs : last unix timestamp to return data for
    </params>
    
    <return>
    retrieved data (json object)
    </return>
    """
    
    url = "https://min-api.cryptocompare.com/data/"
    url += "histo{}".format(histoticks) + "?" + urllib.parse.urlencode(params) 
    res = urllib.request.urlopen(url)
    result = json.loads(res.read().decode('utf-8'))
    return result

if __name__ == "__main__":
    histoticks = "hour"
    params = {
        "fsym": "BTC",
        "tsym": "JPY",
        "limit": "24",
        "e": "bitFlyer"
    }
    result = getRateViaCrypto(histoticks, params)
    for key in result.keys():
        if key != "Data":
            print(key, result[key])
        else:
            print(key)
            for data in result[key]:
                print(data)

# toDataFrame(), toCSV()
CryptoCompareから取得されたデータ中のDataをpandas.DataFrameに変換する。また、csvファイルとして保存する。   
基本的なプロセスは次のようにシンプル。

1. CryptoCopmareから取得する
1. keyをカラム名としてpandas.DataFrameを用意する
1. DataFrame.to_csv()で保存する

In [ ]:
import numpy as np
import pandas as pd
import copy
import datetime

datetimeFmt = "%Y-%m-%dT%H:%M:%S.%f"

def toDataFrame(data, is_datetime=False):
    """toDataFrame(data)
    convert data to a DataFrame object.
    'data' is obtained from CryptoCompare.
    The values in the 'time' are converted to datetime strings 
    if 'is_datetime' == True.
    
    <params>
    data       : data obtained from CryptoCompare (dict object)
    is_datetime: boolean
    </params>
    
    <return>
    a DataFrame object
    </return>
    """
    
    if not isinstance(data, dict):
        raise TypeError
    keys = data["Data"][0].keys()
    
    output = np.zeros((len(data["Data"]), len(keys)), dtype=object)
    for ii, col in enumerate(data["Data"]):
        if is_datetime:
            buff = copy.deepcopy(col)
            datetime1 = datetime.datetime.fromtimestamp(buff["time"])
            buff["time"] = datetime1.strftime(datetimeFmt)
            output[ii] = np.array([buff[key] for key in keys], dtype=object)
        else:
            output[ii] = np.array([col[key] for key in keys])
        
    return pd.DataFrame(output, columns=list(keys))
    

def toCSV(data, fpath, is_datetime=False):
    """toCSV(data, fpath)
    save data to a csv file with the path of 'fpath'.
    'data' must have datasets specified by the 'Data' key.
    The values in the 'time' are converted to datetime strings 
    if 'is_datetime' == True.
    
    <params>
    data       : data obtained from CryptoCompare (dict object)
    fpath      : full (or relative) path to save data to
    is_datetime: boolean
    </params>
    
    <return>
    None
    </return>
    """
    df = toDataFrame(data)
    df.to_csv(fpath)

if __name__ == "__main__":
    histoticks = "hour"
    params = {
        "fsym": "BTC",
        "tsym": "JPY",
        "limit": "5",
        "e": "bitFlyer"
    }
    result = getRateViaCrypto(histoticks, params)
    print(toDataFrame(result, False))
    toCSV(result, "./test.csv")

In [ ]:
!ls

# ローソク足チャートの描画
[こちら](http://nekopuni.holy.jp/2015/01/pythonnysol%E7%82%BA%E6%9B%BF%E9%AB%98%E9%A0%BB%E5%BA%A6%E3%83%87%E3%83%BC%E3%82%BF%E3%81%8B%E3%82%89%E4%BB%BB%E6%84%8F%E3%81%AE%E3%83%AD%E3%82%A6%E3%82%BD%E3%82%AF%E8%B6%B3%E3%83%81%E3%83%A3/)を参照。

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.finance import candlestick_ohlc
%matplotlib inline

df = toDataFrame(result, False)
timestamps = df["time"].as_matrix()


# cand
fig, axes = plt.subplots(figsize=(4, 4))
candlestick_ohlc(axes, df[["time", "open", "high", "low", "close"]].values, width=np.diff(timestamps)[0]*0.8)

# データのダウンロード
google.colab.files.download(filepath)で可能。

In [ ]:
from google.colab import files

files.download("./test.csv")

# データを連続で落としてみる
とはいえ間髪入れずにリクエストするのはよくないので、１分おきにリクエストする。

## DataFrameのテスト
取得したデータを縦結合したいので、そのテスト。   
`pandas.concat(df_list, **kwargs)` を用いる。

In [ ]:
b = pd.concat([df, df], ignore_index=True)
b

とりあえず１日分を落としてみる。   
実際にはlimitに指定した数より１個多めにとっている？

In [ ]:
toTs = datetime.datetime(2018, 5, 6).timestamp()
toTs

In [ ]:
toTs = datetime.datetime(2018, 5, 6).timestamp()
histoticks = "hour"
params = {
    "fsym": "BTC",
    "tsym": "JPY",
    "limit": "23",
    "e": "bitFlyer",
    "toTs":str(toTs)
}
result = getRateViaCrypto(histoticks, params)
for key in result.keys():
    if key != "Data":
        print(key, result[key])
    else:
        print(key)
        for data in result[key]:
            print(data)

30日分を落としてみる。

In [ ]:
toTs = datetime.datetime(2016, 1, 1, 23, 0, 0).timestamp()
histoticks = "hour"
days = 30

params = {
    "fsym": "BTC",
    "tsym": "JPY",
    "limit": str(int(24*days-1)),
    "e": "bitFlyer",
    "toTs":str(toTs)
}
result = getRateViaCrypto(histoticks, params)
df = toDataFrame(result, True)
df.tail(10)

連続して取ってみて、結合してみる。   
ちょっと調べてみたところ、bitFlyerでは2016年01月くらいからがっつりデータが得られそう。   
なのでそこまでのデータを使用する。

In [ ]:
import time

datetimeFmt = "%Y-%m-%dT%H:%M:%S.%f"

toTs_end = datetime.datetime(2016, 1, 1, 0, 0, 0).timestamp()
toTs_last = datetime.datetime(2018, 4, 30, 23, 0, 0).timestamp()
histoticks = "hour"
days = 30
limit = int(24*days-1)

for ii in range(36): # とりあえず３年分に設定
    toTs = toTs_last - ii*days*24*3600
    datetime1 = datetime.datetime.fromtimestamp(toTs)
    print(ii, datetime1.strftime(datetimeFmt))
    if toTs <= toTs_end:
        break
    params = {
        "fsym": "BTC",
        "tsym": "JPY",
        "limit": str(limit),
        "e": "bitFlyer",
        "toTs":str(toTs)
    }
    result = getRateViaCrypto(histoticks, params)
    if ii == 0:
        df = toDataFrame(result, True)
    else:
        df = pd.concat([toDataFrame(result, True), df], ignore_index=True)
    time.sleep(10)
#     if ii == 2:
#         break

In [ ]:
df.head(10)

In [ ]:
df.tail(10)

くしくも2016年01月から2018年04月末までの時間足のデータが入手できたので、保存しておく。

In [ ]:
df.to_csv("./test.csv")

In [ ]:
from google.colab import files

files.download("./test.csv")

# BitFlyerのデータを取得する
BitFlyerで扱われているのは次の銘柄。

* BTC
    + BTC-FX
* ETH
* ETC
* LTC
* BCH
* MONA
* LSK

## bitFlyerのすべての銘柄のデータを落とす

In [ ]:
# brands_interest = ["BTC", "BTCFX", "ETH", "ETC", "LTC", "BCH", "MONA", "LSK"]
# brands_interest = ["JPY", "BTC", "BTC", "BTC", "BTC", "BTC", "BTC", "BTC"]

brands_interest = ["BTC", "ETH", "BCH"] # BTCFX, ETC, LTC, MONA, LSK はない 
brands_interest = ["JPY", "BTC", "BTC"]
dfs = [None] * len(brands)

import time

datetimeFmt = "%Y-%m-%dT%H:%M:%S.%f"

toTs_end = datetime.datetime(2016, 1, 1, 0, 0, 0).timestamp()
toTs_last = datetime.datetime(2018, 4, 30, 23, 0, 0).timestamp()
histoticks = "hour"
days = 30
limit = int(24*days-1)

for ii in range(36): # とりあえず３年分に設定
    toTs = toTs_last - ii*days*24*3600
    datetime1 = datetime.datetime.fromtimestamp(toTs)
    print(ii, datetime1.strftime(datetimeFmt))
    if toTs <= toTs_end:
        break
    for jj in range(len(brands)):
        print(brands[jj])
        params = {
            "fsym": brands[jj],
            "tsym": "JPY",
            "limit": str(limit),
            "e": "bitFlyer",
            "toTs":str(toTs)
        }
        result = getRateViaCrypto(histoticks, params)
        if ii == 0:
            dfs[jj] = toDataFrame(result, True)
        else:
            dfs[jj] = pd.concat([toDataFrame(result, True), dfs[jj]], ignore_index=True)
        time.sleep(10)
#     if ii == 2:
#         break